# Notes
## Questions
* What would this API Client look like with normal Python `.py` files? Would we have it all in one big file or would we split it up a little over multiple, smaller files? Any rough sketch of what that may look like?
* Could we have created a `BaseClient` class for the minimum parts (e.g., client authorization, etc.) and then create a `SpotifyAPIClient(BaseClient)` that inherits from this `BaseClient` for more generalized use? This way a `BaseClient` could be used/inherited inside multiple other API Clients?
## Authorization
* Need to use `client_id` and `client_secret` to retrieve a auth token that will allow us to interact with the Spotify API service.
* Typically authenticate once, retrieve a token which is then attached to your session. During your current session you'll be able to interact/make multiple requests with the service. This allows you to not have to login every time to access your session.
* Handy [Authorization Flow Guide](https://developer.spotify.com/documentation/general/guides/authorization-guide/) from Spotify. We are buidling a **client** to interact with Spotify's API. You have to authenticate how a *user* authenticates with your *client* **and** Spotify at the same time (advanced - not covered fully in this tutorial). We're only using the **Client Credentials** flow, which requires a `Client ID` and `Secret Key` and we'll get in return an `Access Token`. This approach won't give give access to manage private user data, just general data only.
* Our **client** is the application we're building and it (the app) needs to request authorization by sending a `POST` request.
* `base64` is a more secure string format. For non-Base 64 encoded strings, you generally can build something like `client_creds = f"{client_id}:{client_secret}"`. You can encode/decode `str` to bytes using `str.encode()` and `str.encode().decode()`. The Spotify backend API is going to take our base64 encoded `str` (not `bytes`) we pass and then `base64.base64decode()` to verify credentials. Steps to do this:
    1. First take `str` and encode to `bytes` using `str.encode()`
    2. Then encode this bytestring to base64 using `base64.b64encode(bytestring)`
    3. Next, decode this base64 `bytes` so it's back to `str` using `client_creds_b64.decode()` (**not** `b64decode()`).
* The token response data we get after making the `POST` request to authorize our client has the `access_token` and the `expires_in` (seconds) details. We can use `datetime` and the `timedelta(seconds=expires_in)` to create an easy `expires` `datetime.datetime` object for tracking. We can check whether our token expired and to fetch a new one: `did_expire: bool = expires < now`.
## Base API Client Class
* Once we have our `access_token` then we can use it. However, the `expires` and `access_tokens` are **state-like** items, so they will change over time. So, better would be to create a `BaseClient` **class** that can adjust/respond to these state changes in order to make our API calls through the class (and its methods) rather than just through simple functions like we did in `Auth.ipynb`. We're going to turn our functions into class methods.
* Create our first class `SpotifyAPI(object)` with some general-purpose variables (nothing hard-coded since we want to reuse this for various sessions). We can pass the `client_id` and `client_secret` to the `__init__` function. The `super().__init__(*args, **kwargs)` allows us to call the class our class is inheriting from *directly*. Currently we're not doing that but in the future if we wanted to then having this line allows us to do that. 
* ?? We set `client_id` and `client_secret` to `None` by default for the class. However, we need to **update** them...?? Aren't these already available though since we set them inside the __init__ function? I think so but this makes it cleaner so we don't have to `{self.client_id}`? Not sure though as he mentioned "updating" these values...
    * I *believe* we have to update the class variables within these methods because when we initialize/create a new instance of the class, we only pass in `client_id` and `client_secret` (based on our `__init__`). However, the other class variables (`access_token`, `expires`, etc.) are defaulted to `None` so our class methods need to update these values e.g, `self.access_token = access_token` (once we have retrieved the true `access_token`).
* **Refactor** Could refactor the base class to have a method that handles the response data instead of all within the current `perform_authorization()` method.
* **Refactor** Need to improve how we get the `access_token` because when it does expire, ideally it will go fetch another one automatically by re-performing this authorization function again.
* Eventually going to take this base class and expand it to allow for something like `client.search` (`spotify.search`).
## Use the Access Token
* Following the [Web API](https://developer.spotify.com/documentation/web-api/reference/) docs.
* Once we get our `access_token` after `perform_authorization()`, the `token_type: "bearer"` instead of `Basic`. So, the initial `token_headers` header of `Authorization` had a `Basic` type. After we perform authorization, we receive `token_type: "bearer"`. This means we use `bearer` token moving forward.
* We're essentially authorized to interact with the API via our new Client app. That means for each request we make to say, search for various songs or albums, we need to have this `bearer` token attached. Also, for Spotify specifically, we build up a request to query but we pass in all the `data` *directly* inside the `endpoint`/`lookup_url` we pass to `requests.get()`. 
    * To create this `lookup_url` string, we're going to use `from urllib.parse import urlencode` to help.
* We now authenicated and worked with the API via our base class (client app). We have a working example of using the `search` endpoint of the API. However, we don't want to copy/paste the request code each time. Better would be to refactor/improve our `client`/`spotify` app (class) by adding some more helper/utility/convenience class methods to perform these searches, auto-refresh/fetch our `access_token` when it expires, and even allow accessing other endpoints (`artists`, `/v1/albums/{id}`, etc.). 
## Resource-Enabled Client
* We now authenicated and worked with the API via our base class (client app). We have a working example of using the `search` endpoint of the API. However, we don't want to copy/paste the request code each time. Better would be to refactor/improve our `client`/`spotify` app (class) by adding some more helper/utility/convenience class methods to perform these searches, auto-refresh/fetch our `access_token` when it expires, and even allow accessing other endpoints (`artists`, `/v1/albums/{id}`, etc.). 
* Need to add a `get_access_token()` method and `search()` method. Our `perform_authorization()` method doesn't actually return the `access_token` -- it's currently just stored in `self.access_token` so better would be to create a new method specific to getting it and then using it within our authorization based on whether it was successfully retrieved.
* Add an additional attribute to our class (client) e.g., `def get_album()`, `def get_artist()` etc. So far we're only using the `Search` endpoint provided by Spotify's API. There [are a lot of other](https://developer.spotify.com/documentation/web-api/reference/search/search/) endpoints such as `Albums`, `Artists` etc. we could access.
* Since there is an obvious pattern to accessing the `Albums` and `Artists` endpoints, let's create another method `def get_resource()` that is a more generalized method where we can pass the `resource_type` (e.g., `album` or ` artist`) and the `_id`. 
* Again, each request we make must be authorized and once we're (our client) is authorized, we have to attach/add an `"Authorization": "Bearer {access_token}"` `header` within our `request`. Since this `header` is required for every request, better would be to refactor and create an additional helper/convenience method `get_resource_header` (or could be called `get_access_header`, etc.).
* **Challenge** Could expand by making a sub-class (another object?) for artists and albums themselves so we can get even more data about them.


In [23]:
import typing as t
import requests
import base64
import datetime
from urllib.parse import urlencode

In [24]:
# Snag creds from developer.spotify.com dashboard
client_id: str
client_secret: str

In [39]:
# Declare our first class with helper methods
class SpotifyAPIClient(object):
    access_token: str = None
    access_token_expires = datetime.datetime.now()  # or None
    access_token_did_expire: bool = True
    client_id: str = None
    client_secret: str = None
    token_url: str = "https://accounts.spotify.com/api/token"

    def __init__(self, client_id, client_secret, *args, **kwargs):
        # Call super() so we can call any class we inherit from directly!
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self) -> str:
        """
        Returns a Base 64 encoded string (not bytes!)
        """
        # Need to declare our client_id and client_secret vars
        # The self obj has them but need to update within this
        # method's scope:
        # NOTE ?? Aren't these already available though since we
        # set them inside the __init__ function? I think so but
        # this makes it cleaner so we don't have to {self.client_id}??
        client_id: str = self.client_id
        client_secret: str = self.client_secret

        # Update the client_id and client_secret values
        if client_id is None or client_secret is None:
            raise Exception("You must set client_id and client_secret.")

        # Build the credentials str required by Spotify:
        client_creds: str = f"{client_id}:{client_secret}"
        client_creds_bytes: bytes = client_creds.encode()
        client_creds_b64: bytes = base64.b64encode(client_creds_bytes)
        # NOTE We must .decode() back to 'str' type instead of 'bytes'
        client_creds_b64_decoded: str = client_creds_b64.decode()

        return client_creds_b64_decoded

    def get_token_headers(self) -> t.Dict:
        """
        Pass 'Authorization' header with b64 encoded creds
        "Authorization": "Basic <base64 encoded client_id:client_secret>"
        """
        client_creds_b64_decoded: str = self.get_client_credentials()

        token_headers: t.Dict = {
            "Content-Type": "application/x-www-form-urlencoded",
            "Authorization": f"Basic {client_creds_b64_decoded}"
        }

        return token_headers
    
    def get_token_data(self) -> t.Dict:
        token_data: t.Dict = {
            "grant_type": "client_credentials",
        }

        return token_data

    def perform_authorization(self) -> bool:
        """
        Extracts the access token and other variables for authorizing
        the client app with Spotify API. It uses helper methods to get
        client credentials, convert to Base 64, get token headers and
        token data. 
        """
        # TODO Re-run authorization if access_token expires
        # Use helper methods to retrieve components for sending request
        token_url: str = self.token_url
        token_data: t.Dict = self.get_token_data()
        token_headers: t.Dict = self.get_token_headers()

        # Now we have everything for authentication so it's time to make the POST request
        r = requests.post(token_url, data=token_data, headers=token_headers)

        if r.status_code not in range(200, 299):
            raise Exception(f"Could not authenticate client: {status_code}")
            # return False

        # Let's store the access token, expires in (seconds), etc.
        data: t.Dict = r.json()
        now = datetime.datetime.now()
        access_token: str = data['access_token']
        expires_in: float = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)

        # Update our class variables with these updated values
        self.access_token = access_token
        self.access_token_expires = expires  # datetime.datetime obj
        self.access_token_did_expire = expires < now  # refetch the token if True

        return True  # authorization successful

    def get_access_token(self) -> str:
        """
        Retrieve the authorized access token for our client app.
        """
        token: str = self.access_token
        expires = self.access_token_expires  # datetime
        now = datetime.datetime.now()

        if expires < now:
            # Retrieve by re-running this exact function!
            self.perform_authorization()
            return self.get_access_token()
        elif token is None:
            self.perform_authorization()
            return self.get_access_token()
        return token
    
    def get_resource_headers(self) -> t.Dict:
        """
        Retrieve the authorized access headers for our client app.
        Specifically, retrieve the Authorization header with the
        Bearer token. This is the header we want/need to pass for
        all of our resources/endpoints we access.
        """
        access_token: str = self.get_access_token()
        headers: t.Dict = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers



    def get_resource(self, lookup_id: str, resource_type: str = "albums", api_version: str = "v1") -> t.Dict:
        """
        Generalized method for retrieving a single artist or album,
        based on the resource type and id value. Must use an
        authorized client in order to access resource type.

        Params:
            _id = ID of the resource
            resource_type = Spotify API endpoint/resource to access
        """
        assert resource_type.lower() in ["artists", "albums"]
        base_url: str = "https://api.spotify.com"
        endpoint: str = f"{base_url}/{api_version}/{resource_type}/{lookup_id}"
        headers: t.Dict = self.get_resource_headers()

        # We have the pieces let's now build our request
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            print(f"Failed request: {r.status_code} : {endpoint}")
            return {}
        return r.json()

    
    def get_album(self, _id: str):
        """
        Search for a single album via Spotify's Albums endpoint:
        https://api.spotify.com/v1/albums/{id}
        """
        # === OLD reference only ===
        # base_url: str = "https://api.spotify.com/"
        # endpoint: str = f"{base_url}/v1/albums/{_id}"
        # r = requests.get(endpoint)
        # if r.status_code not in range(200, 299):
        #     return {}
        # return r.json()

        return self.get_resource(lookup_id=_id, resource_type="albums")

    def get_artist(self, _id: str):
        """
        Search for a single artist via Spotify's Artists endpoint:
        https://api.spotify.com/v1/artists/{id}
        """
        return self.get_resource(lookup_id=_id, resource_type="artists")


        
    def search(self, query: str, search_type: str = 'artist'):
        """
        Search the Spotify API for various songs, artists,
        albums, etc.

        Params:
            query- Query string that gets url encoded
            search_type - Default='artist'. Type of search to perform in Spotify
        """
        assert search_type.lower() in ["artist", "track", "playlist", "album", "show", "episode"]
        # Use our helper method to retrieve resource headers
        # which has the Bearer token inside
        headers: t.Dict = self.get_resource_headers()
        endpoint: str = "https://api.spotify.com/v1/search"

        # We need to pass our 'data' within the url string we send our GET request to
        # To make this easier we use urlencode
        # NOTE Could consider having 'type' be a List or Dict for more options
        data: str = urlencode({
            "q": query,
            "type": search_type.lower()  # "Track" -> "track"
        })

        # Let's build/concat our 'lookup_url' for use. Don't forget '?' symbol!
        lookup_url: str = f"{endpoint}?{data}"

        # Now that we have the bare minimum requirements, let's make a request
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            print(f"Failed request: {r.status_code} : {lookup_url}")
            return {}
        return r.json()

In [40]:
# Now let's test out our base class (client app)
# client = SpotifyAPI(client_id, client_secret)
# Could generalize this client for multiple services
spotify = SpotifyAPIClient(client_id, client_secret)

In [44]:
# Test out our new search() method
spotify.search(query="A lannister always pays his debts", search_type="track")

  'PA',
      'PE',
      'PH',
      'PL',
      'PS',
      'PT',
      'PY',
      'QA',
      'RO',
      'RS',
      'RU',
      'SA',
      'SE',
      'SG',
      'SI',
      'SK',
      'SV',
      'TH',
      'TN',
      'TR',
      'TW',
      'UA',
      'US',
      'UY',
      'VN',
      'XK',
      'ZA'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/2JgKlCS2glC37XDWreeCy2'},
     'href': 'https://api.spotify.com/v1/albums/2JgKlCS2glC37XDWreeCy2',
     'id': '2JgKlCS2glC37XDWreeCy2',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b27342a847dc8ca888c02ca4488f',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e0242a847dc8ca888c02ca4488f',
       'width': 300},
      {'height': 64,
       'url': 'https://i.scdn.co/image/ab67616d0000485142a847dc8ca888c02ca4488f',
       'width': 64}],
     'name': 'Cinematic Chillout - Relaxing Soundtracks Music',
     'release_date': '2020

In [42]:
# Let's test out our new get_album, get_artist methods
spotify.get_artist("1w5Kfo2jwwIPruYS2UWh56")  # Pearl Jam

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1w5Kfo2jwwIPruYS2UWh56'},
 'followers': {'href': None, 'total': 5907203},
 'genres': ['alternative rock', 'grunge', 'permanent wave', 'rock'],
 'href': 'https://api.spotify.com/v1/artists/1w5Kfo2jwwIPruYS2UWh56',
 'id': '1w5Kfo2jwwIPruYS2UWh56',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/8a7775f16a99af5d353d4eb31ec0ccf908a6a6d1',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/7d40296db84567ff1f76c6bb3a561de53bccadb3',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/4733adb065f2e5c637bf7362413e9d2a9e330310',
   'width': 160}],
 'name': 'Pearl Jam',
 'popularity': 79,
 'type': 'artist',
 'uri': 'spotify:artist:1w5Kfo2jwwIPruYS2UWh56'}

In [45]:
spotify.get_artist("1hCkSJcXREhrodeIHQdav8")  # Ramin Djawadi

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
 'followers': {'href': None, 'total': 426151},
 'genres': ['german soundtrack',
  'scorecore',
  'soundtrack',
  'video game music'],
 'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
 'id': '1hCkSJcXREhrodeIHQdav8',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/3192cec3d51144c7a061848927f89983835c5480',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/d33c97f0483a37ecf2d8309f99b1abaa1e499fd9',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/553f6b2aa9cb60ac1353f8efbd9edf3542332e8f',
   'width': 160}],
 'name': 'Ramin Djawadi',
 'popularity': 73,
 'type': 'artist',
 'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}

In [46]:
spotify.get_album("2JgKlCS2glC37XDWreeCy2")

kets': ['AD',
     'AE',
     'AL',
     'AR',
     'AT',
     'AU',
     'BA',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'BY',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
     'GR',
     'GT',
     'HK',
     'HN',
     'HR',
     'HU',
     'ID',
     'IE',
     'IL',
     'IN',
     'IS',
     'IT',
     'JO',
     'JP',
     'KW',
     'KZ',
     'LB',
     'LI',
     'LT',
     'LU',
     'LV',
     'MA',
     'MC',
     'MD',
     'ME',
     'MK',
     'MT',
     'MX',
     'MY',
     'NI',
     'NL',
     'NO',
     'NZ',
     'OM',
     'PA',
     'PE',
     'PH',
     'PL',
     'PS',
     'PT',
     'PY',
     'QA',
     'RO',
     'RS',
     'RU',
     'SA',
     'SE',
     'SG',
     'SI',
     'SK',
     'SV',
     'TH',
     'TN',
     'TR',
     'TW',
     'UA',
     'US',
     'UY',
     'VN',
     'X